In [8]:
import numpy as np

In [9]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(f"{type(data)}은(는) 지원하지 않습니다.")

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

    def backward_r(self):
        f = self.creator
        if f is not None:
            x = f.input
            x.grad = f.backward(self.grad)
            x.backward()

In [10]:
from abc import ABCMeta, abstractmethod

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    else:
        return x

class Function(metaclass=ABCMeta):
    def __call__(self, inputs, *args, **kwargs):
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    @abstractmethod
    def forward(self, x):
        pass

    @abstractmethod
    def backward(self, gy):
        pass

In [11]:
class Square(Function):
    def forward(self, x):
        return x ** 2

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

def square(x):
    return Square()(x)

In [12]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

def exp(x):
    return Exp()(x)

In [13]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(np.array(x.data - eps))
    x1 = Variable(np.array(x.data + eps))
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (eps * 2)

In [14]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))

y.backward()
print(x.grad)

TypeError: 'Variable' object is not iterable

In [15]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y, )
    def backward(self, gy):
        pass

xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
print(y.data)

5


In [1]:
import sys
module_path = "../../"
sys.path.append(module_path)

In [2]:
import numpy as np
from dezero.core_simple import Variable

x = Variable(np.array(1.0))
print(x)

variable(1.0)


TypeError: unsupported operand type(s) for +: 'Variable' and 'int'